<a href="https://colab.research.google.com/github/hyunaeee/upstage/blob/main/ensembel_method_ranksum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Let's begin!




### 0. Prepare

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# !pip install -r /content/drive/MyDrive/fastcampus/nlp_project/code/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 53.4 MB/s eta 0:00:00
   ━━━━

In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, DataCollatorForLanguageModeling
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from trl import SFTTrainer, SFTConfig

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset, DatasetDict

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel, PeftConfig

In [ ]:
config_data = {
    "general": {
        "data_path": "/content/drive/MyDrive/fastcampus/nlp_project/data/",
        "model_name": "beomi/Llama-3-Open-Ko-8B", # meta-llama/Llama-2-7b-hf -> beomi/Llama-3-Open-Ko-8B
        # "t5_model_id": "eenzeenee/t5-small-korean-summarization",
        "output_dir": "/content/drive/MyDrive/fastcampus/nlp_project/output/",
    },
    "tokenizer": {
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#',
                           '#DateOfBirth#', '#SSN#', '#CardNumber#', '#CarNumber#', '#Email#',
                           '#Person#', '#Person4#', '#Person5#', '#Person6#', '#Person7#']
    },
    "training": {
        "overwrite_output_dir": True,
        "max_input_length": 512,
        "max_target_length": 128,
        "num_train_epochs": 15,
        "batch_size": 8,
        "learning_rate": 1e-5, # 1e-5, optuna: 3e-5
        "per_device_train_batch_size": 10, # 50
        "per_device_eval_batch_size": 2, # 32
        "warmup_ratio": 0.1,
        "weight_decay": 0.01, # 0.01, optuna: 0.005
        "lr_scheduler_type": 'cosine',
        "metric_for_best_model": "loss",
        "optim": 'adamw_torch', # adamw_torch, optuna: adamw_hf
        "gradient_accumulation_steps": 3, # 1
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 6, # 5
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_steps": 10,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 200, # 100
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3, # 3
        "early_stopping_threshold": 0, # 0.001
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    "inference": {
        "ckt_path": "/content/drive/MyDrive/fastcampus/nlp_project/output/", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "/content/drive/MyDrive/fastcampus/nlp_project/prediction/",
    }
}

In [ ]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "/content/drive/MyDrive/fastcampus/nlp_project/code/config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 1. Load Dataset

- RankSum의 실행 단계 설명:

> 문장 평가: 각 문장은 길이(length), 위치(position), TF-IDF 점수(tfidf)와 같은 여러 특성으로 평가됩니다. 이 특성들은 문장이 얼마나 중요한지를 판단하는 데 사용됩니다.

> 가중치 적용: 각 특성에 대해 설정된 가중치를 통해, 어떤 특성이 더 중요한지를 반영합니다. 예를 들어, TF-IDF 점수가 더 중요하다면 해당 가중치를 높게 설정할 수 있습니다.

> 총점 계산: 각 문장의 특성 점수에 가중치를 곱하여 총점을 계산합니다. 이 총점은 문장의 중요성을 나타내며, 이후 요약에서 포함될 문장을 결정하는 데 사용됩니다.

> 문장 정렬: 계산된 총점에 따라 문장을 내림차순으로 정렬하여 가장 중요한 문장을 선택합니다.

> 요약 생성: 상위 문장들을 선택하여 최종 요약을 생성합니다.

- RankSum의 장점:

> 유연성: 사용자가 각 특성의 가중치를 조정할 수 있어, 특정 도메인이나 요구 사항에 맞게 요약의 품질을 개선할 수 있습니다.

> 다양한 특성 반영: 단순히 TF-IDF 점수만 고려하는 것이 아니라, 문장의 길이와 위치와 같은 다양한 요소를 함께 고려함으로써 보다 균형 잡힌 요약을 생성합니다.

> 병렬 처리: joblib를 사용한 병렬 처리를 통해 CPU의 성능을 극대화하여 문장 점수 계산 속도를 향상시킵니다. 이는 대량의 데이터 처리 시 유리합니다.

> 직관적인 접근법: 각 문장의 중요성을 직관적으로 이해할 수 있는 방식으로 평가하기 때문에, 결과를 해석하기 쉽습니다.

> 효율성: RankSum 방식은 상대적으로 간단하면서도 효과적인 방법으로, 복잡한 모델이나 알고리즘 없이도 좋은 품질의 요약을 생성할 수 있습니다.

- RankSum을 사용한 배경:

> 두 모델의 요약 결과가 상이하기에 하나의 요약으로 재작성해보고 싶은 생각이 들었다.

> kbart로 단순 재요약: 33.1824(mistral model) + 49.4275(gpt model) => 30.2376(점수 하락)

> RankSum 요약(파인튜닝 이전): 33.1824(mistral model) + 49.4275(gpt model) => 35.9586(점수 보정)

> RankSum 요약(파인튜닝 이후): 35.9586 => 37.8467(점수 상승)

In [ ]:
# 저장된 config 파일을 불러옵니다.
config_path = "/content/drive/MyDrive/fastcampus/nlp_project/code/config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '/content/drive/MyDrive/fastcampus/nlp_project/data/',
             'model_name': 'beomi/Llama-3-Open-Ko-8B',
             'output_dir': '/content/drive/MyDrive/fastcampus/nlp_project/output/'},
 'inference': {'ckt_path': '/content/drive/MyDrive/fastcampus/nlp_project/output/',
               'result_path': '/content/drive/MyDrive/fastcampus/nlp_project/prediction/'},
 'tokenizer': {'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#PhoneNumber#',
                                  '#Address#',
                                  '#PassportNumber#',
                                  '#DateOfBirth#',
                                  '#SSN#',
                                  '#CardNumber#',
                                  '#CarNumber#',
                                  '#Email#',
                                  '#Person#',
                          

In [ ]:
# Save results as a CSV file
result_path = loaded_config['inference']['result_path']

In [ ]:
output_file_path1 = os.path.join(result_path, "output_mistral_finetuned.csv")

# CSV 파일 불러오기
df1 = pd.read_csv(output_file_path1)


In [ ]:
output_file_path2 = os.path.join(result_path, "output_gpt2.csv")

# CSV 파일 불러오기
df2 = pd.read_csv(output_file_path2)


In [ ]:
# test data의 구조와 내용을 확인합니다.
data_path = loaded_config['general']['data_path']
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
test_df = pd.read_csv(os.path.join(data_path,'test.csv'))

In [ ]:
# Generate summaries
summaries1 = df1['summary'].tolist()
summaries2 = df2['summary'].tolist()
all_summaries = []
dash_line = '-'.join('' for x in range(100))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from joblib import Parallel, delayed

nltk.download('punkt')
nltk.download('punkt_tab')

def calculate_sentence_score(sentence, index, tfidf_matrix):
    """주어진 문장의 점수를 계산합니다."""
    score = {
        'index': index,
        'text': sentence,
        'length': len(sentence.split()),
        'position': 1 / (index + 1),  # 위치 점수
        'tfidf': np.sum(tfidf_matrix[index])  # TF-IDF 점수
    }
    return score

def calculate_sentence_scores(sentences, tfidf_matrix):
    """cpu의 속도를 개선하고자 병렬로 계산합니다."""
    scores = Parallel(n_jobs=-1)(
        delayed(calculate_sentence_score)(sentence, i, tfidf_matrix)
        for i, sentence in enumerate(sentences)
    )
    return scores

def normalize_scores(scores):
    """점수를 정규화 계산합니다."""
    features = ['length', 'position', 'tfidf']
    for feature in features:
        values = np.array([score[feature] for score in scores])
        max_score = values.max()
        min_score = values.min()
        if max_score > min_score:
            for score in scores:
                score[feature] = (score[feature] - min_score) / (max_score - min_score)
        else:
            for score in scores:
                score[feature] = 0
    return scores

def rank_sum(scores, weights):
    total_scores = np.zeros(len(scores))

    for feature, weight in weights.items():
        total_scores += np.array([score[feature] * weight for score in scores])

    for i, score in enumerate(scores):
        score['total'] = total_scores[i]

    return sorted(scores, key=lambda x: x['total'], reverse=True)

def generate_summary_ranksum(text1, text2, num_sentences=3):
    # 두 텍스트 결합
    full_text = text1 + " " + text2

    # 문장 분리
    sentences = sent_tokenize(full_text)

    # TF-IDF 계산
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences).toarray()

    # 문장 점수 계산 (병렬 처리)
    scores = calculate_sentence_scores(sentences, tfidf_matrix)

    # 점수 정규화
    normalized_scores = normalize_scores(scores)

    # 가중치 설정 (필요에 따라 조정 가능)
    weights = {
        'length': 0.1,
        'position': 0.4,
        'tfidf': 0.5
    }

    # RankSum 적용
    ranked_sentences = rank_sum(normalized_scores, weights)

    # 상위 문장 선택
    top_sentences = sorted(ranked_sentences[:num_sentences], key=lambda x: x['index'])

    # 요약 생성
    summary = ' '.join(sentence['text'] for sentence in top_sentences)

    return summary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
for idx in range(len(summaries1)):
    # Generate the combined summary
    final_summary = generate_summary_ranksum(summaries1[idx], summaries2[idx])

    print(f"\n##{idx}")
    print('\n')
    print(dash_line)
    print('\n')
    print("Mistral Summary:")
    print(summaries1[idx])
    print('\n')
    print("GPT Summary:")
    print(summaries2[idx])
    print('\n')
    print(dash_line)
    print('\n')
    print("Generated Summary:")
    print(final_summary)
    all_summaries.extend([final_summary])
    print('\n')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
#Person1#은 자리에서 잠을 자려고 한다. #Person2#은 전화기를 놓아 둬야 한다고 말한다. #Person1#은 깨어나는 시간을 놓치지 않도록 한다.


GPT Summary:
#Person1#과 #Person2#는 전화기를 뽑고, #Person1#은 #Person2#가 깨울 것이라고 생각합니다.


---------------------------------------------------------------------------------------------------


Generated Summary:
#Person2#은 전화기를 놓아 둬야 한다고 말한다. #Person1#은 깨어나는 시간을 놓치지 않도록 한다. #Person1#과 #Person2#는 전화기를 뽑고, #Person1#은 #Person2#가 깨울 것이라고 생각합니다.



##279


---------------------------------------------------------------------------------------------------


Mistral Summary:
#Person1#은 선생님과 중국 음식과 서양 음식 중 어떤 것을 드시고 싶으신가요?
#Person2#는 중국 음식을 먹고 싶어요.
#Person1#은 이 세트 메뉴를 드실건가요, 아니면 식당에서 식사하실 건가요?
#Person2#는 여기서 먹을게요.
#Person1#은 앞에 있는 테이블을 내려주세요. 그게 더 편합니다.
#Person2#는 아, 감사합니다. 정말 친절하시네요.
#Person1#은 어떤 걸 마시고 싶으신가요, 우유, 차, 커피, 오렌지 주스, 아니면


GPT Summary:
#Person1#이 #Person2#에게 중국 음식을 추천하고, #Person2#가 2인분 식사를 주문하는 데 도움을 줍니다.


--------------------------------------------------------

In [ ]:
# Save results as a CSV file
result_path = loaded_config['inference']['result_path']
if not os.path.exists(result_path):
    os.makedirs(result_path)



# Create a DataFrame with 'fname' and 'generated_summary'
output_csv = pd.DataFrame({
    'fname': test_df['fname'],
    'summary':  all_summaries
})

# Save the DataFrame to CSV
output_file_path = os.path.join(result_path, "output_finetuned_ensemble.csv")
output_csv.to_csv(output_file_path, index=False)

# Log to W&B (assuming you have initialized wandb)
wandb.log({"final_summaries": wandb.Table(dataframe=output_csv)})

print(f"Generated summaries saved to {output_file_path}")

Generated summaries saved to /content/drive/MyDrive/fastcampus/nlp_project/prediction/output.csv
